## Define evaluate function

In [1]:
import epics
import time
import numpy as np

def do_measurement(inputs):
    """use pyepics to set input pvs, wait a few seconds and then do a measurement"""

    mean_charge = 2.61e2  # gun charge in pC
    charge_dev = 0.1  # factional charge deviation

    # set values
    for name, val in inputs.items():
        epics.caput(name, val)

    # wait
    time.sleep(10.0)

    data = {}

    # calculate total losses
    hard_cblm_indexes = range(13, 46)
    hard_loss_PVS = [f"CBLM:UNDH:{ele}75:I1_LOSS" for ele in hard_cblm_indexes]

    data["TMITH"] = epics.caget("BPMS:LI30:201:TMITCUH1H") / 1e9

    losses = epics.caget_many(hard_loss_PVS)

    data["TOTAL_HARD_LOSSES"] = np.sum(losses)

    # get averaged pulse intensity for HXR
    data["NEG_HXR_AVG_INTENSITY"] = -epics.caget(
        "EM2K0:XGMD:HPS:AvgPulseIntensity"
    )

    data["time"] = time.time()

    return data

## Define VOCS

In [2]:
import pandas as pd
from xopt import VOCS
filename = "../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')

variable_names = [f"QUAD:LI26:{i}01:BCTRL" for i in range(2,10)]

vocs = VOCS(
    variables={ele: VARIABLE_RANGES[ele] for ele in variable_names},
    objectives={"NEG_HXR_AVG_INTENSITY": "MINIMIZE"},
)
vocs

VOCS(variables={'QUAD:LI26:201:BCTRL': [10.2, 12.3], 'QUAD:LI26:301:BCTRL': [-12.5, -5.7], 'QUAD:LI26:401:BCTRL': [8.8, 12.2], 'QUAD:LI26:501:BCTRL': [-4.5, -3.7], 'QUAD:LI26:601:BCTRL': [11.2, 12.8], 'QUAD:LI26:701:BCTRL': [-14.5, -13.0], 'QUAD:LI26:801:BCTRL': [12.2, 14.1], 'QUAD:LI26:901:BCTRL': [-10.0, -6.8]}, constraints={}, objectives={'HXR_AVG_INTENSITY': 'MAXIMIZE'}, constants={}, observables=[])

## Define generator, evaluator and Xopt object

In [3]:
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt import Xopt


evaluator = Evaluator(function=do_measurement)
generator = UpperConfidenceBoundGenerator(
    vocs=vocs, turbo_controller="optimize"
)
# for experiments
generator.model_constructor.use_low_noise_prior = False

X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)
X


            Xopt
________________________________
Version: 2.0a1+104.g9e7df7c9.dirty
Data size: 0
Config as YAML:
dump_file: null
evaluator:
  function: __main__.do_measurement
  function_kwargs: {}
  max_workers: 1
  vectorized: false
generator:
  beta: 2.0
  computation_time: null
  fixed_features: null
  max_travel_distances: null
  model: null
  model_constructor:
    covar_modules: {}
    mean_modules: {}
    name: standard
    trainable_mean_keys: []
    use_low_noise_prior: false
  n_candidates: 1
  n_monte_carlo_samples: 128
  name: upper_confidence_bound
  numerical_optimizer:
    max_iter: 2000
    n_raw_samples: 20
    n_restarts: 20
    name: LBFGS
  turbo_controller:
    batch_size: 1
    best_value: null
    center_x: null
    dim: 8
    failure_counter: 0
    failure_tolerance: 4
    length: 0.25
    length_max: 2.0
    length_min: 0.0078125
    minimize: false
    scale_factor: 2.0
    success_counter: 0
    success_tolerance: 4
    tkwargs:
      dtype: torch.float64


## Evaluate initial point(s) and view data

In [4]:
initial_points = {}
X.evaluate_data(initial_points)

KeyError: 'QUAD:LI26:201:BCTRL'

In [ ]:
X.data

## Run optimization

In [ ]:
for n in range(5):
    print(n)
    X.step()